# QMSolver: Infinite Square Well Limit Validation

This notebook validates `FDSolver` accuracy by solving a finite square well with very large depth and comparing to analytical infinite square well solutions.

In [1]:
import numpy as np
from scipy import constants

from qmsolver.tise import FDSolver
from qmsolver.potentials import FiniteSquareWellPotential

In [2]:
well_depth_ev = 1e12  # Very large well depth in electron volts
well_width_nm = 1.0  # Well width in nanometers

# Convert to SI units
well_depth_joules = well_depth_ev * constants.e  # Joules
well_width_meters = well_width_nm * 1e-9  # meters

# Spatial domain in meters
x_min_m = -3e-9
x_max_m = 3e-9

print("Physical parameters:")
print(f"Well depth: {well_depth_ev:.0e} eV = {well_depth_joules:.8e} J")
print(f"Well width: {well_width_nm} nm = {well_width_meters:.1e} m")
print(f"Spatial domain: [{x_min_m:.1e}, {x_max_m:.1e}] m")

Physical parameters:
Well depth: 1e+12 eV = 1.60217663e-07 J
Well width: 1.0 nm = 1.0e-09 m
Spatial domain: [-3.0e-09, 3.0e-09] m


In [3]:
solver = FDSolver(steps=4_000, x_min=x_min_m, x_max=x_max_m, n_lowest=5)

# Set physical constants in SI units
solver.h_bar = constants.hbar  # Reduced Planck's constant in J⋅s
solver.m = constants.m_e  # Electron mass in kg

potential = FiniteSquareWellPotential(
    x_grid=solver.x_grid,
    well_depth=well_depth_joules,
    well_width=well_width_meters,
)
solver.potential_generator = potential

solver.solve()
solver.output()

****************************************

-> 5 lowest energy states:

      🔒   E(0)  = -1.60217663399939821925e-07  (bound)
      🔒   E(1)  = -1.60217663399759350967e-07  (bound)
      🔒   E(2)  = -1.60217663399458574862e-07  (bound)
      🔒   E(3)  = -1.60217663399037493608e-07  (bound)
      🔒   E(4)  = -1.60217663398496133676e-07  (bound)

****************************************


# Analytical Infinite Square Well Energies

The analytical energies for an infinite square well are given by:
$$E_n = n^2 \frac{\hbar^2 \pi^2}{2mL^2}$$

where:
- $n = 1, 2, 3, \dots$ (quantum number)
- $L$ is the well width
- $\hbar$ is the reduced Planck constant
- $m$ is the particle mass

In [4]:
E_lowest_ev = np.array(solver.E_lowest) / constants.e
print("\nEnergies in electron volts:")
for i, energy in enumerate(E_lowest_ev):
    infinite_square_well_energy = (
        (i + 1) ** 2
        * constants.hbar**2
        * np.pi**2
        / (2 * solver.m * well_width_meters**2)
    ) / constants.e

    renormalized_energy = energy + well_depth_ev

    error = (
        100
        * abs(renormalized_energy - infinite_square_well_energy)
        / infinite_square_well_energy
    )

    print(
        f"E({i}) = {renormalized_energy:.8f} eV | E_ISW({i}): {infinite_square_well_energy:.8f} eV | Error: {error:.8f} %"
    )


Energies in electron volts:
E(0) = 0.37548828 eV | E_ISW(0): 0.37603016 eV | Error: 0.14410569 %
E(1) = 1.50195312 eV | E_ISW(1): 1.50412065 eV | Error: 0.14410569 %
E(2) = 3.37927246 eV | E_ISW(2): 3.38427146 eV | Error: 0.14771268 %
E(3) = 6.00744629 eV | E_ISW(3): 6.01648259 eV | Error: 0.15019248 %
E(4) = 9.38635254 eV | E_ISW(4): 9.40075405 eV | Error: 0.15319530 %


# Conclusion

Based on the comparison of the `FDSolver` results with the analytical infinite square well solutions, the numerical method provides reliable results for bound state problems. In this case, the numerical results match the expected analytical values with an error of approximately 0.15% using a 4,000-step grid resolution. Higher grid resolution generally reduces the error, with 10,000 steps achieving errors below 0.1%.

However, the very deep finite well is only an approximation to the infinite square well, which introduces additional errors beyond those from the numerical method itself. Therefore, the error rates reported here represent a conservative estimate of `FDSolver`'s accuracy. Nevertheless, this analysis demonstrates that `FDSolver` is a reliable tool for numerically solving bound state problems.